<a href="https://colab.research.google.com/github/aymericvie/Covid19_coevolution/blob/main/Covid_coevolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
# imports
import math
import random
import numpy as np
random.seed(9)

# Fixed parameters
virus_size = 10
initial_virus_population = 2
baseR = round(random.uniform(2,3),2)
#mut = np.sort(np.random.normal(0,1,size=virus_size))
mut = a = np.arange(-1,1+1/10,2/virus_size)
print(mut)

policy_size = 6
population_policy_size = 10

tmax = 15
crossover_rate_policy = 0.9
mutation_rate_virus = 0.0001
mutation_rate_policy = 0.1

# Parameters for non pharmaceutical interventions (NPIs) from Haug et al (2020)
npi = [round(random.uniform(-0.36,-0.18),2), round(random.uniform(-0.24,-0.16),2), round(random.uniform(-0.24,-0.08),2), round(random.uniform(-0.18,-0.08),2), round(random.uniform(-0.18,-0.04),2), round(random.uniform(-0.18,0),2)]

# Initial sampling for viruses
population_viruses = np.zeros((initial_virus_population,virus_size))
print("initial pop virus")
print(population_viruses)

# Initial sampling for policies
population_policies = np.zeros((population_policy_size,policy_size))
population_virus_size = initial_virus_population


print("initial pop polici")
print(population_policies)

# determine efficacy of population viruses and policies

t = 0
# Initial results
distribution_virus_genome = np.mean(population_viruses,0)
distribution_policy_genome = np.mean(population_policies,0)
virusR_history = []
policyR_history = []


while t < tmax+1:

  virusR = np.zeros((population_virus_size,1))
  i = 0
  while i < population_virus_size:
    virusR[i,0] = baseR + np.sum(population_viruses, axis = 1)[i]
    i += 1
  #print(virusR)


  policyR = np.zeros((population_policy_size,1))
  i = 0
  while i < population_policy_size:
    policyR[i,0] = np.dot(population_policies[i,],npi)
    i += 1

  #print(policyR)

  # evaluate coevolution fitness evaluation for policies, with tournament(3) selection
  policy_fitness = np.zeros((population_policy_size,1))

  i = 0
  while i < population_policy_size:
    a = random.randint(0,population_virus_size-1)
    b = random.randint(0,population_virus_size-1)
    c = random.randint(0,population_virus_size-1)
    policy_fitness[i,0] = (1/3)*(virusR[a,0] + virusR[b,0] + virusR[c,0]) + policyR[i,0]
    i += 1
  #print(policy_fitness)
  adjusted_policy_fitness = 1/(1+policy_fitness)
  #print(adjusted_policy_fitness)
  policy_selection = adjusted_policy_fitness / np.sum(adjusted_policy_fitness)
  #print(policy_selection)

  cumulative_policy_selection = np.copy(policy_selection)
  i = 1
  while i < len(cumulative_policy_selection):
    cumulative_policy_selection[i,0] += cumulative_policy_selection[i-1,0]
    i += 1

  i = 0
  while i < population_policy_size:
    a = random.uniform(0,1)
    b = 0
    while a > cumulative_policy_selection[b]:
        b += 1

    a = random.uniform(0,1)
    c = 0
    while a > cumulative_policy_selection[c]:
        c += 1

  # Do crossover between elements b and c

    offspring1 = np.zeros((1, policy_size))
    offspring2 = np.zeros((1, policy_size))

    n = random.uniform(0,1)
    if n <= crossover_rate_policy:

      cpoint = random.randint(0,policy_size)
      j = 0
      while j < cpoint:
        offspring1[0,j] = population_policies[b,j]
        offspring2[0,j] = population_policies[c,j]
        j += 1
      while j < policy_size:
        offspring1[0,j] = population_policies[c,j]
        offspring2[0,j] = population_policies[b,j]
        j += 1
      #print(offspring1)
      #print(offspring2)

    elif n > crossover_rate_policy:
      offspring1 = np.copy(population_policies[b,])
      offspring2 = np.copy(population_policies[c,])
      #print(offspring1)
      #print(offspring2)
      #print(offspring2.shape)
      offspring2 = offspring2.reshape((1,policy_size))
      offspring1 = offspring1.reshape((1,policy_size))
      #print(offspring2.shape)
    k = 0
    while k < policy_size:
      n = random.uniform(0,1)
      if n <= mutation_rate_policy:
        if offspring1[0,k] == 0:
          offspring1[0,k] = 1
        elif offspring1[0,k] == 1:
          offspring1[0,k] = 0
      k += 1

    k = 0
    while k < policy_size:
      n = random.uniform(0,1)
      if n <= mutation_rate_policy:
        if offspring2[0,k] == 0:
          offspring2[0,k] = 1
        elif offspring2[0,k] == 1:
          offspring2[0,k] = 0
      k += 1

    population_policies = np.vstack([population_policies,offspring1])
    population_policies = np.vstack([population_policies,offspring2])
    i += 2

  #print("new pop polici")
  population_policies = population_policies[population_policy_size:, :]
  #print(population_policies)


  # operate virus transmission and mutation
  i = 0
  while i < population_virus_size:
    j = 0
    # determine the number of self replications for each virus 
    p = (virusR[i,0]+np.mean(policyR)) - math.floor(virusR[i,0]+np.mean(policyR))
    # print("p"+str(p))
    # print("l"+str(l))
    if p <= 0.5:
      l = random.uniform(0,1)
      if l <= p:
        m = math.floor(virusR[i,0]+np.mean(policyR))
      if l > p:
        m = math.ceil(virusR[i,0]+np.mean(policyR))
      # print(virusR[i,0]+np.mean(policyR))
      # print(m)
    if p > 0.5:
      l = random.uniform(0,1)
      if l > 1-p:
        m = math.floor(virusR[i,0]+np.mean(policyR))
      if l <= 1-p:
        m = math.ceil(virusR[i,0]+np.mean(policyR))
      # print(virusR[i,0]+np.mean(policyR))
      # print(m)      
    while j < m:
      candidate_virus = np.copy(population_viruses[i,])
      k = 0
      while k < virus_size:
        n = random.uniform(0,1)
        if n <= mutation_rate_virus:
          if candidate_virus[k] == 0:
            candidate_virus[k] = 1
          elif candidate_virus[k] == 1:
            candidate_virus[k] = 0
        k += 1
      #print(candidate_virus)
      population_viruses = np.vstack([population_viruses,candidate_virus])
      j += 1
    i += 1

  #print("new pop virus")
  population_viruses = population_viruses[population_virus_size:, :]
  #print(population_viruses)
  population_virus_size = population_viruses.shape[0]
  print("Virus pop size "+str(population_virus_size))

  # Save results
  if t > 0:
    distribution_virus_genome = np.vstack([distribution_virus_genome,np.mean(population_viruses,0)])
    distribution_policy_genome = np.vstack([distribution_policy_genome,np.mean(population_policies,0)])

  virusR_history.append(round(np.mean(virusR),2))
  policyR_history.append(round(np.mean(policyR),2))

  print("Current time: " + str(t)+"/"+str(tmax))
  t += 1



[-1.00000000e+00 -8.00000000e-01 -6.00000000e-01 -4.00000000e-01
 -2.00000000e-01 -2.22044605e-16  2.00000000e-01  4.00000000e-01
  6.00000000e-01  8.00000000e-01  1.00000000e+00]
initial pop virus
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
initial pop polici
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Virus pop size 6
Current time: 0/15
Virus pop size 16
Current time: 1/15
Virus pop size 42
Current time: 2/15
Virus pop size 118
Current time: 3/15
Virus pop size 326
Current time: 4/15
Virus pop size 930
Current time: 5/15
Virus pop size 2692
Current time: 6/15
Virus pop size 7802
Current time: 7/15
Virus pop size 7856
Current time: 8/15
Virus pop size 8636
Current time: 9/15
Virus pop size 9641
Current time: 10/15
Virus pop size 10370
Current time: 11/15
Virus pop size 11394
Current time: 12/15
Vir

In [106]:
print(distribution_virus_genome)
print(distribution_policy_genome)
print(virusR_history)
print(policyR_history)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.07526882e-03
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.11441308e-03 0.00000000e+00 0.00000000e

In [94]:
math.ceil(1.769)

2